In [0]:
#@title #Consent Form { run: "auto", vertical-output: true, display-mode: "form" }
#@markdown 1 - I have had the details of the study explained to me and willingly consent to take part. My questions have been answered to my satisfaction and I understand that I may ask further questions at any time.
_1 = False #@param {type:"boolean"} 
#@markdown 2 - I understand that I will remain anonymous and that all the information given will be used in the study only.
_2 = False #@param {type:"boolean"} 
#@markdown 3 - I understand that all information given will be kept confidential. Anonymous data might be shared with other researchers. This would never be linked to my name.
_3 = False #@param {type:"boolean"}
#@markdown 4 - I understand how the results of the study will be used. We will write up the results for publication. No names will be used.
_4 = False #@param {type:"boolean"}
#@markdown 5 - I confirm that I am over 16 years of age
_5 = False #@param {type:"boolean"}
#@markdown  6 - I would like to take part in the study.
_6 = False #@param {type:"boolean"}


#@markdown 7 - Please tell us your age.
Age = 0 #@param {type:"slider", min:0, max:100, step:1}
#@markdown #--------------------------------------------------------------------------

#@markdown #**After filling out the consent form, click on the play button on the left.**
consent = _1 and _2 and _3 and _4 and _5 and _6



if consent:
  !git clone https://github.com/PedroFerreiradaCosta/stylegan-encoder.git
  import os
  os.chdir("stylegan-encoder")
  %tensorflow_version 1.x
  import pickle
  from ipywidgets import widgets
  import PIL.Image
  import numpy as np
  import dnnlib
  import dnnlib.tflib as tflib
  import config
  from encoder.generator_model import Generator
  %load_ext autoreload
  %autoreload 2

  import matplotlib.pyplot as plt
  #%matplotlib inline
  %matplotlib inline
  from IPython.display import clear_output
  !pip install bayesian-optimization
  import bayes_opt
  import tensorflow as tf
  import warnings
  from sklearn.gaussian_process.kernels import Matern, WhiteKernel
  warnings.filterwarnings("ignore", category=DeprecationWarning)
  print('')

  ###############################################################################

  # path to model code and weight
  path_model = 'https://drive.google.com/uc?id=1I1vyhKJhoQul3ryIrHcTlpYycPAhWooI'

  """ create tf session """
  yn_CPU_only = False

  if yn_CPU_only:
      config = tf.ConfigProto(device_count = {'GPU': 0}, allow_soft_placement=True)
  else:
      config = tf.ConfigProto(allow_soft_placement=True)
      config.gpu_options.allow_growth = True

  sess = tf.InteractiveSession(config=config)

  try:
      with dnnlib.util.open_url(path_model, 'rb') as file:
          G, D, Gs = pickle.load(file)
  except FileNotFoundError:
      print('before running the code, download pre-trained model to project_root/asset_model/')
      raise
  generator = Generator(Gs, batch_size=1, randomize_noise=False)

  ###############################################################################

  def generate_image(latent_vector):
      latent_vector = latent_vector.reshape((1, 18, 512))
      generator.set_dlatents(latent_vector)
      img_array = generator.generate_images()[0]
      img = PIL.Image.fromarray(img_array, 'RGB')
      #return img
      return img.resize((256, 256))


  def move_and_show(latent_vector, direction, coeffs):
      fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
      for i, coeff in enumerate(coeffs):
          new_latent_vector = latent_vector.copy()
          new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
          ax[i].imshow(generate_image(new_latent_vector))
          ax[i].set_title('Coeff: %0.1f' % coeff)
      [x.axis('off') for x in ax]
      plt.show()

  def plot_sampled_face(latent_vector, direction1, direction2, x, y, i):
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plt.figure(figsize=(10,10))
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + x*direction1)[:8]
    new_latent_vector[:8] = (new_latent_vector + y*direction2)[:8]
    plt.imshow(generate_image(new_latent_vector))
    plt.title('Iteration: {}'.format(i))
    plt.axis('off')
    plt.show()

  ###############################################################################

  # Loading already learned latent directions
  smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
  gender_direction = np.load('ffhq_dataset/latent_directions/gender.npy')
  age_direction = np.load('ffhq_dataset/latent_directions/age.npy')

  os.mkdir('raw_images')
  os.mkdir('latent_representations')
  os.mkdir('aligned_images')
  os.mkdir('generated_images')

  ###############################################################################

  from IPython.display import display, Javascript
  from google.colab.output import eval_js
  from base64 import b64decode

  def take_photo(filename='raw_images/photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
      ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return filename

  print('')  
  print("Press Capture below to take a photo")
  print("Try to avoid accessories like glasses or hats.")

  from IPython.display import Image
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
    
    # Show the image which was just taken.
    #display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

  ###############################################################################

  # 1) Extract and align faces from images
  !rmdir raw_images/.ipynb_checkpoints/
  # Images to be processed should be added to the folder 'raw_images'
  !python align_images.py raw_images/ aligned_images/
  # if downloading dlib face landmarks takes more than 1/2 min, rerun code
  print('')
  print("This step will take a while.\n In the meantime, complete the form below. To be added")
  print('')

  # 2) Find latent representation of aligned images
  !python encode_images.py aligned_images/ generated_images/ latent_representations/ --iterations 1000

  # src_dir - directory with images
  # generated_images_dir - directory for storing generated images
  # dlatent_dir - directory for storing dlatent representations
  # batch_size - df - 1 - for generator and perceptual model
  # image_size - df - 256 - Size of images for perceptual model
  # lr - df 1 - learning rate for perc. model
  # iterations - df - 1000 - opetimizations steps per batch 
  # randomize_noise-  df - False - add noise to dlatents during optimization

  ###############################################################################

  # Substitute here for the latent you want to use
  latent_vector =np.load('latent_representations/photo_01.npy')

  ###############################################################################

  from bayes_opt import BayesianOptimization, UtilityFunction
  from ipywidgets import interact, interactive, fixed, interact_manual
  import ipywidgets as widgets
  from IPython import display
  import time

  def f(rating):
    optimizer.register(next_point_to_probe, rating)
    
    return rating

  pbounds = {'x': (-1, 1), 'y': (-1, 1)}
  optimizer = BayesianOptimization(
      f=None,
      pbounds=pbounds,
      verbose=2,
      random_state=42,
  )
  kernel = 1.0 * Matern(length_scale=0.20, length_scale_bounds=(0.05, 1), nu=2.5)\
          + WhiteKernel(noise_level=50, noise_level_bounds=(1e-10, 0.5))
  optimizer.set_gp_params(kernel=kernel)
  
  utility = UtilityFunction(kind="ucb", kappa=2, xi=0)

  ###############################################################################

  init_points = 5
  n_iter = 20

  #Rate_the_face = 1.7 #@param {type:"slider", min:0, max:10, step:0.1}

  for _ in range(init_points + n_iter):

      if _ < init_points:
          next_point_to_probe = {'x': np.random.uniform(pbounds['x'][0], pbounds['x'][1]),
                                  'y': np.random.uniform(pbounds['y'][0], pbounds['y'][1])}

          x, y = next_point_to_probe.values()
      else:
          next_point_to_probe = optimizer.suggest(utility)
          x, y = next_point_to_probe.values()


      # Here I show the image generated by the new latent
      plot_sampled_face(latent_vector, smile_direction, age_direction, x*2, y*2, _)


      #print("Point in the space is:", next_point_to_probe)
      
      #wait = input('Click Enter on the box below when a rating has been chosen.\n')
      wait = input('How much does this image look like you?Rate it in the box below from 0 to 10, 10 being just like you.\n You can use decimals (e.g. 8.7).\n')
      rating = float(wait)
      optimizer.register(next_point_to_probe, float(rating))
      # print(direction)
      # while direction == 'b':
      #     time.sleep(0.1)

  maximum, params = optimizer.max.values()
  x_max,y_max = params.values()
  print('')
  print('Optimum smile position: {}'.format(x_max))
  print('Optimum age position: {}'.format(y_max))

  ###############################################################################
  from vis import plot_bo
  from itertools import product

  x = np.linspace(-1, 1, 50).reshape(-1, 1)
  y = np.linspace(-1, 1, 50).reshape(-1, 1)

  x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
  y_obs = np.array([[res["params"]["y"]] for res in optimizer.res])
  z_obs = np.array([res["target"] for res in optimizer.res])

  x1x2 = np.array(list(product(x, y)))
  X0p, X1p = x1x2[:, 0].reshape(50, 50), x1x2[:, 1].reshape(50, 50)

  X = (np.array([x_obs.ravel(), y_obs.ravel()])).T
  np.save('values',np.array([x_obs.ravel(), y_obs.ravel(), z_obs.ravel()]).T)
  
  #optimizer._gp.fit(X, z_obs)
  mu, std = optimizer._gp.predict(x1x2.reshape(-1, 2), return_std=True)
  np.save('mu',mu)
  np.save('std',std)
  ###############################################################################
  p_name = str(Age)
  # libraries to be imported 
  import smtplib 
  from email.mime.multipart import MIMEMultipart 
  from email.mime.text import MIMEText 
  from email.mime.base import MIMEBase 
  from email import encoders 
    
  fromaddr = "face.fitting@gmail.com"
  toaddr = "face.fitting@gmail.com"
    
  # instance of MIMEMultipart 
  msg = MIMEMultipart() 
    
  # storing the senders email address   
  msg['From'] = fromaddr 
    
  # storing the receivers email address  
  msg['To'] = toaddr 
    
  # storing the subject  
  msg['Subject'] = p_name
    
  # string to store the body of the mail 
  body = "Optimum smile position: {} \n Optimum age position: {} ".format(x_max, y_max)
    
  # attach the body with the msg instance 
  msg.attach(MIMEText(body, 'plain')) 
    
  # open the file to be sent  
  filename = "latent_vector.npy"
  attachment = open("./latent_representations/photo_01.npy", "rb") 
  p = MIMEBase('application', 'octet-stream') 
  p.set_payload((attachment).read()) 
  encoders.encode_base64(p) 
  p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  msg.attach(p) 

  filename = "mean_space.npy"
  attachment = open("./mu.npy", "rb") 
  q = MIMEBase('application', 'octet-stream') 
  q.set_payload((attachment).read()) 
  encoders.encode_base64(q) 
  q.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  msg.attach(q) 

  filename = "variance_space.npy"
  attachment = open("./std.npy", "rb") 
  r = MIMEBase('application', 'octet-stream') 
  r.set_payload((attachment).read()) 
  encoders.encode_base64(r) 
  r.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  msg.attach(r) 

  filename = "value_target.npy"
  attachment = open("./values.npy", "rb") 
  v = MIMEBase('application', 'octet-stream') 
  v.set_payload((attachment).read()) 
  encoders.encode_base64(v) 
  v.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  msg.attach(v) 

  # creates SMTP session 
  s = smtplib.SMTP('smtp.gmail.com', 587) 
    
  # start TLS for security 
  s.starttls() 
    
  # Authentication 
  s.login(fromaddr, "facefitting94") 
    
  # Converts the Multipart msg into a string 
  text = msg.as_string() 
    
  # sending the mail 
  s.sendmail(fromaddr, toaddr, text) 
    
  # terminating the session 
  s.quit() 

  clear_output()
else:
  print('You must complete the consent form before starting.')

.

.

.

.

.

.

.

In [0]:
#@title How does your Space look like? Play this cell. {display-mode: "form"}



plot_bo(mu, std, x, y, 'Smile', 'Age')
# wait = input('Press Enter to clear')
# clear_output()

In [0]:
#@title What is the face you rated the highest? {display-mode: "form"}
# This code will be hidden when the notebook is loaded.
def plot_final_face(latent_vector, direction1, direction2, x, y):
  display.clear_output(wait=True)
  display.display(plt.gcf())
  plt.figure(figsize=(10,20))
  plt.subplot(121)
  plt.title('Face "0"')
  plt.imshow(generate_image(latent_vector))
  new_latent_vector = latent_vector.copy()
  new_latent_vector[:8] = (latent_vector + x*direction1)[:8]
  new_latent_vector[:8] = (new_latent_vector + y*direction2)[:8]
  plt.axis('off')
  plt.subplot(122)
  plt.title('Face rated the highest')
  plt.imshow(generate_image(new_latent_vector))
  plt.axis('off')
  plt.show()
  print('Shift in emotion: {}'.format(x))
  print('Shift in age: {}'.format(y))

plot_final_face(latent_vector, smile_direction, age_direction, x_max*2, y_max*2)

# wait = input('Press enter to clear')
# clear_output()

In [0]:
#@title To try again, first click play on this cell. {display-mode: "form"}

!kill -9 -1

print('You can now play again the first cell.')
